In [31]:
import pickle
import pickle
import numpy as np
import os
import sys
sys.path.append('../simulation')
import study
from sklearn import preprocessing
import time

In [18]:
def get_phi(exp,pi,baseline_features,responsivity_features):
        all_data = []
        steps=[]
      

        ##might add pi to the user's history
        for user_id,user_data in exp.population.items():
            history = user_data.history
            history_keys = sorted({k:v for k,v in history.items() if v['decision_time']})
          
            for hk in history_keys:
            
                h = history[hk]
               
                v = [1]
                v.extend([h[i] for i in baseline_features])
                v.append(pi*1)
                v.extend([pi*h[i] for i in responsivity_features])
                action = h['action']
                if action<0:
                    action=0
                
                v.append((action-pi)*1)
                v.extend([(action-pi)*h[i] for i in responsivity_features])
                v.append(action)
                v.append(float(user_id))
                v.append(float(h['study_day']))
                all_data.append(v)
                steps.append(h['steps'])
    
    
        return all_data,steps

In [6]:
root =  '../../../../Volumes/dav/HeartSteps/pooling_rl_shared_data/temporary_files/'
def get_experiment():
    with open('{}simulated_history.pkl'.format(root),'rb') as f:
        return pickle.load(f)

In [11]:
exp = get_experiment()

In [19]:
data,steps = get_phi(exp,.6,['weather','location','ltps'],['weather','location','ltps'])

In [83]:
def new_standardize(X,y):
    new_x = [x[:-2] for x in X]
    new_x = preprocessing.scale(np.array(new_x))
    to_return = np.zeros((len(X),len(X[0])))
    for i in range(len(X)):
        #temp=np.zeros(len(X[i]))
        to_return[i][:-2]=new_x[i]
        to_return[i][-2]=X[i][-2]
        to_return[i][-1]=X[i][-1]
    
    return to_return,preprocessing.scale(np.array(y))

In [86]:
def new_standardize_faster(X,y):
    new_x = [x[:-2] for x in X]
    new_x = preprocessing.scale(np.array(new_x))
    to_return = np.zeros((len(X),len(X[0])))
    for i in range(len(X)):
        temp=new_x[i].tolist()
        temp.extend(X[i][-2:])
        to_return[i]=np.array(temp)
    return to_return,preprocessing.scale(np.array(y))

In [87]:
times_one = []
times_two = []
for i in range(100):
    start = time.time()
    tx = new_standardize(data,steps)
    end = time.time()
    times_one.append(end-start)
    start = time.time()
    tx = new_standardize_faster(data,steps)
    end = time.time()
    times_two.append(end-start)

In [88]:
print(np.array(times_one).mean())
print(np.array(times_two).mean())

0.0258988356590271
0.035096099376678465


In [47]:
start = time.time()
tx = new_standardize_faster(data,steps)
end = time.time()
print(end-start)

0.04175996780395508


In [53]:
def create_H(num_baseline_features,num_responsivity_features):
    ##for now have fixed random effects size one
    
    random_effect_one = [1]
    random_effect_two = [1]
    
    row_one = []
    
    
    
    #np.transpose(np.array([[1]+[0 for i in range(context_len)]+[0,0,0,0],[0]+[0 for i in range(context_len)]+[1,0,1,0]]))

array([ 0.        ,  0.9898399 ,  0.28742215, -0.49941618,  0.        ,
        0.9898399 ,  0.28742215, -0.49941618, -0.36301921, -1.06834361,
       -0.44676152,  0.4020273 , -0.36301921,  2.        , 67.        ])